In [27]:
import numpy as np
import torch
import pandas as pd
import json

In [15]:
whisper_embeds_path = '../data/whsiper_embeds/whisper_embeds.pt'

In [39]:
whisper_embeds = torch.load(whisper_embeds_path)
whisper_embeds.keys()

dict_keys(['fleurs-it_it-test.pt', 'fleurs-lt_lt-test.pt', 'fleurs-hu_hu-test.pt', 'fleurs-nb_no-test.pt', 'fleurs-fr_fr-test.pt', 'fleurs-tr_tr-test.pt', 'fleurs-ln_cd-test.pt', 'fleurs-az_az-test.pt', 'fleurs-uz_uz-test.pt', 'fleurs-kk_kz-test.pt', 'fleurs-ar_eg-test.pt', 'fleurs-km_kh-test.pt', 'fleurs-el_gr-test.pt', 'fleurs-te_in-test.pt', 'fleurs-sw_ke-test.pt', 'fleurs-hi_in-test.pt', 'fleurs-sk_sk-test.pt', 'fleurs-mk_mk-test.pt', 'fleurs-sn_zw-test.pt', 'fleurs-fi_fi-test.pt', 'fleurs-mr_in-test.pt', 'fleurs-ca_es-test.pt', 'fleurs-fa_ir-test.pt', 'fleurs-as_in-test.pt', 'fleurs-hy_am-test.pt', 'fleurs-ro_ro-test.pt', 'fleurs-lb_lu-test.pt', 'fleurs-mn_mn-test.pt', 'fleurs-oc_fr-test.pt', 'fleurs-bg_bg-test.pt', 'fleurs-et_ee-test.pt', 'fleurs-vi_vn-test.pt', 'fleurs-sd_in-test.pt', 'fleurs-ur_pk-test.pt', 'fleurs-ps_af-test.pt', 'fleurs-es_419-test.pt', 'fleurs-ne_np-test.pt', 'fleurs-lv_lv-test.pt', 'fleurs-bn_in-test.pt', 'fleurs-tg_tj-test.pt', 'fleurs-ka_ge-test.pt', 'fle

In [40]:
whisper_embeds['tira-asr-orig-test.pt'].shape

torch.Size([1280])

In [41]:
# taken from https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists
cosine = lambda v1, v2: np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

In [42]:
tira_embeds = whisper_embeds['tira-asr-orig-test.pt']
tira_embeds.shape

torch.Size([1280])

In [43]:
cos_sim = []
langs = list(whisper_embeds.keys())
for lang in langs:
    cos_sim.append(cosine(whisper_embeds[lang], tira_embeds))
cos_sim[:5], max(cos_sim)

([-0.015650928, -0.011643961, -0.011675436, -0.009698147, -0.009488348], 1.0)

In [44]:
df=pd.DataFrame({'embeds_file': langs, 'cos_sim': cos_sim})
df.head()

,embeds_file,cos_sim
0,fleurs-it_it-test.pt,-0.015651
1,fleurs-lt_lt-test.pt,-0.011644
2,fleurs-hu_hu-test.pt,-0.011675
3,fleurs-nb_no-test.pt,-0.009698
4,fleurs-fr_fr-test.pt,-0.009488


In [45]:
lang_code = df['embeds_file'].apply(lambda s: s.replace('fleurs-','').replace('-test.pt',''))
df['lang_code']=lang_code
df.head()

,embeds_file,cos_sim,lang_code
0,fleurs-it_it-test.pt,-0.015651,it_it
1,fleurs-lt_lt-test.pt,-0.011644,lt_lt
2,fleurs-hu_hu-test.pt,-0.011675,hu_hu
3,fleurs-nb_no-test.pt,-0.009698,nb_no
4,fleurs-fr_fr-test.pt,-0.009488,fr_fr


In [52]:
lang_code_path = '../meta/language_codes.json'
with open(lang_code_path) as f:
    lang_code_dict = json.load(f)
lang_code_dict['tira']='tira-asr-orig'
lang_code_dict

{'english': 'en_us',
 'german': 'de_de',
 'spanish': 'es_419',
 'russian': 'ru_ru',
 'korean': 'ko_kr',
 'french': 'fr_fr',
 'japanese': 'ja_jp',
 'portuguese': 'pt_br',
 'turkish': 'tr_tr',
 'polish': 'pl_pl',
 'catalan': 'ca_es',
 'dutch': 'nl_nl',
 'arabic': 'ar_eg',
 'swedish': 'sv_se',
 'italian': 'it_it',
 'indonesian': 'id_id',
 'hindi': 'hi_in',
 'finnish': 'fi_fi',
 'vietnamese': 'vi_vn',
 'hebrew': 'he_il',
 'ukrainian': 'uk_ua',
 'greek': 'el_gr',
 'malay': 'ms_my',
 'czech': 'cs_cz',
 'romanian': 'ro_ro',
 'danish': 'da_dk',
 'hungarian': 'hu_hu',
 'tamil': 'ta_in',
 'norwegian': 'nb_no',
 'thai': 'th_th',
 'urdu': 'ur_pk',
 'croatian': 'hr_hr',
 'bulgarian': 'bg_bg',
 'lithuanian': 'lt_lt',
 'maori': 'mi_nz',
 'malayalam': 'ml_in',
 'welsh': 'cy_gb',
 'slovak': 'sk_sk',
 'telugu': 'te_in',
 'persian': 'fa_ir',
 'latvian': 'lv_lv',
 'bengali': 'bn_in',
 'serbian': 'sr_rs',
 'azerbaijani': 'az_az',
 'slovenian': 'sl_si',
 'kannada': 'kn_in',
 'estonian': 'et_ee',
 'macedonia

In [53]:
code_to_lang = {v:k for k, v in lang_code_dict.items()}
lang_names = df['lang_code'].apply(lambda s: code_to_lang[s])
df['lang']=lang_names
df.head()

,embeds_file,cos_sim,lang_code,lang
0,fleurs-it_it-test.pt,-0.015651,it_it,italian
1,fleurs-lt_lt-test.pt,-0.011644,lt_lt,lithuanian
2,fleurs-hu_hu-test.pt,-0.011675,hu_hu,hungarian
3,fleurs-nb_no-test.pt,-0.009698,nb_no,norwegian
4,fleurs-fr_fr-test.pt,-0.009488,fr_fr,french


In [54]:
df.sort_values('cos_sim', ascending=False)

,embeds_file,cos_sim,lang_code,lang
79,tira-asr-orig-test.pt,1.000000,tira-asr-orig,tira
66,fleurs-pl_pl-test.pt,-0.008499,pl_pl,polish
29,fleurs-bg_bg-test.pt,-0.009038,bg_bg,bulgarian
67,fleurs-nl_nl-test.pt,-0.009137,nl_nl,dutch
4,fleurs-fr_fr-test.pt,-0.009488,fr_fr,french
...,...,...,...,...
7,fleurs-az_az-test.pt,-0.017138,az_az,azerbaijani
38,fleurs-bn_in-test.pt,-0.018377,bn_in,bengali
45,fleurs-mi_nz-test.pt,-0.018417,mi_nz,maori
6,fleurs-ln_cd-test.pt,-0.021068,ln_cd,lingala


In [55]:
lang_to_embeds_file = {}
add_pair = lambda r: lang_to_embeds_file.update({r['lang']:r['embeds_file']})
df.apply(add_pair, axis=1)
lang_to_embeds_file['tira']='tira-asr-orig-test.pt'
lang_to_embeds_file

{'italian': 'fleurs-it_it-test.pt',
 'lithuanian': 'fleurs-lt_lt-test.pt',
 'hungarian': 'fleurs-hu_hu-test.pt',
 'norwegian': 'fleurs-nb_no-test.pt',
 'french': 'fleurs-fr_fr-test.pt',
 'turkish': 'fleurs-tr_tr-test.pt',
 'lingala': 'fleurs-ln_cd-test.pt',
 'azerbaijani': 'fleurs-az_az-test.pt',
 'uzbek': 'fleurs-uz_uz-test.pt',
 'kazakh': 'fleurs-kk_kz-test.pt',
 'arabic': 'fleurs-ar_eg-test.pt',
 'khmer': 'fleurs-km_kh-test.pt',
 'greek': 'fleurs-el_gr-test.pt',
 'telugu': 'fleurs-te_in-test.pt',
 'swahili': 'fleurs-sw_ke-test.pt',
 'hindi': 'fleurs-hi_in-test.pt',
 'slovak': 'fleurs-sk_sk-test.pt',
 'macedonian': 'fleurs-mk_mk-test.pt',
 'shona': 'fleurs-sn_zw-test.pt',
 'finnish': 'fleurs-fi_fi-test.pt',
 'marathi': 'fleurs-mr_in-test.pt',
 'catalan': 'fleurs-ca_es-test.pt',
 'persian': 'fleurs-fa_ir-test.pt',
 'assamese': 'fleurs-as_in-test.pt',
 'armenian': 'fleurs-hy_am-test.pt',
 'romanian': 'fleurs-ro_ro-test.pt',
 'luxembourgish': 'fleurs-lb_lu-test.pt',
 'mongolian': 'fleur

In [ ]:
067050634 sep 16 7:45am 1100 union str 3rd floor

In [63]:
def compare_langs(lang1, lang2):
    embed1 = whisper_embeds[lang_to_embeds_file[lang1]]
    embed2 = whisper_embeds[lang_to_embeds_file[lang2]]
    return cosine(embed1, embed2)

compare_langs('welsh', 'georgian')

0.9772243